In [1]:
%load_ext autoreload
%autoreload 2

from pprint import pprint
from sklearn.metrics import classification_report
from utils.inference import compute_metrics


DATASET_NAME = "toxigen"
LABEL_SPACE = ["non-toxic", "toxic"]
MODEL_NAME = "deberta_large"
SEED = 42
POOLER = "mean_with_attention"
LAYER = 24

def evaluate(y_true, y_pred, is_multiclass: bool, prefix: str='test'):
    # Print some metrics
    testset_perfm = compute_metrics(
        y_true=y_true, y_pred=y_pred, is_multiclass=is_multiclass, prefix=prefix
    )
    pprint(testset_perfm)
    print(classification_report(y_true=y_true, y_pred=y_pred))


In [2]:
## Load Embeddings
from utils.io import (
    load_dataset_from_hf,
    load_labels_at_split,
    load_embeddings,
)
import numpy as np
train_embeddings = load_embeddings(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    split="train",
    pooler=POOLER,
    layer=LAYER
)

eval_embeddings = load_embeddings(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    split="eval",
    pooler=POOLER,
    layer=LAYER
)

test_embeddings = load_embeddings(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    split="test",
    pooler=POOLER,
    layer=LAYER
)

train_eval_embeddings = np.vstack([train_embeddings, eval_embeddings])

## Load Datasets and Labels
dataset = load_dataset_from_hf(dataset=DATASET_NAME)
train_labels = load_labels_at_split(dataset, "train")
eval_labels = load_labels_at_split(dataset, "eval")
train_eval_labels = np.concatenate([train_labels, eval_labels])
test_labels = load_labels_at_split(dataset, "test")

from datasets import DatasetDict, concatenate_datasets
train_eval_dataset = concatenate_datasets([dataset["train"], dataset["eval"]])
dataset_dict = DatasetDict(
    {"train": train_eval_dataset, "test": dataset["test"]}
)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/samsoup/.cache/huggingface/token
Login successful


/home/samsoup/anaconda3/envs/wrapperbox/lib/python3.9/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


In [3]:
import pickle
from sklearn.linear_model import LogisticRegression


l2 = 1
logit_clf = LogisticRegression(penalty="l2", C= 1 / l2)
logit_clf.fit(train_eval_embeddings, train_eval_labels)
predictions = logit_clf.predict(test_embeddings)


evaluate(
    y_pred=predictions, 
    y_true=test_labels, 
    is_multiclass=np.unique(test_labels).size > 2
)

# Path to save the model
model_path = 'LogisticRegression.pkl'

# Save the trained model to a .pkl file
with open(model_path, 'wb') as file:
    pickle.dump(logit_clf, file)

print(f"Model saved to {model_path}")

{'test_accuracy': 0.825531914893617,
 'test_f1': 0.6893939393939394,
 'test_precision': 0.7459016393442623,
 'test_recall': 0.6408450704225352}
              precision    recall  f1-score   support

           0       0.85      0.91      0.88       656
           1       0.75      0.64      0.69       284

    accuracy                           0.83       940
   macro avg       0.80      0.77      0.78       940
weighted avg       0.82      0.83      0.82       940

Model saved to LogisticRegression.pkl


In [4]:
from classifiers.RandomClassifier import RandomClassifier


random_clf = RandomClassifier()
random_clf.fit(train_eval_embeddings, train_eval_labels)
predictions = random_clf.predict(test_embeddings)

evaluate(
    y_pred=predictions, 
    y_true=test_labels, 
    is_multiclass=np.unique(test_labels).size > 2
)

{'test_accuracy': 0.48829787234042554,
 'test_f1': 0.36959370904325034,
 'test_precision': 0.29436325678496866,
 'test_recall': 0.4964788732394366}
              precision    recall  f1-score   support

           0       0.69      0.48      0.57       656
           1       0.29      0.50      0.37       284

    accuracy                           0.49       940
   macro avg       0.49      0.49      0.47       940
weighted avg       0.57      0.49      0.51       940



In [5]:
from lightgbm import LGBMModel

num_labels = np.unique(test_labels).size
objective='binary' if num_labels == 2 else 'multiclass'
clf = LGBMModel(
    objective=objective,
    num_classes=1 if objective=="binary" else num_labels,
    learning_rate=1.0,      # Set to 1.0 for a single tree
    n_estimators=1,         # Build only one tree
    min_child_samples=20,   # minimum samples in leaf 
    n_jobs= -1,
    random_state=42
)
clf.fit(train_eval_embeddings, train_eval_labels)

prediction_probas = clf.predict(test_embeddings)
if objective == "binary":
    predictions = (prediction_probas >= 0.5).astype(int) # threshold using 0.5
else:
    predictions = np.argmax(prediction_probas, axis=1)

from pprint import pprint
from sklearn.metrics import classification_report
from utils.inference import compute_metrics

# Print some metrics
testset_perfm = compute_metrics(
    y_true=test_labels, y_pred=predictions, is_multiclass=objective=="multiclass", prefix="test"
)
pprint(testset_perfm)
print(classification_report(y_true=test_labels, y_pred=predictions))

# import pickle
# # Save model to file
# model_filename = 'DecisionTree.pkl'
# with open(model_filename, 'wb') as f:
#     pickle.dump(clf, f)


[LightGBM] [Info] Number of positive: 2178, number of negative: 6782
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.088054 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 8960, number of used features: 1024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.243080 -> initscore=-1.135865
[LightGBM] [Info] Start training from score -1.135865
{'test_accuracy': 0.8202127659574469,
 'test_f1': 0.6817325800376648,
 'test_precision': 0.7327935222672065,
 'test_recall': 0.6373239436619719}
              precision    recall  f1-score   support

           0       0.85      0.90      0.87       656
           1       0.73      0.64      0.68       284

    accuracy                           0.82       940
   macro avg       0.79      0.77      0.78       940
weighted avg       0.82      0.82      0.82       940



In [6]:
from classifiers.KMeansClassifier import KMeansClassifier


clf = KMeansClassifier(algorithm='elkan', init='k-means++', n_clusters=2,random_state=42)
clf.fit(train_eval_embeddings, train_eval_labels)
predictions = clf.predict(test_embeddings)

evaluate(
    y_pred=predictions, 
    y_true=test_labels, 
    is_multiclass=np.unique(test_labels).size > 2
)

import pickle
# Save model to file
model_filename = 'LMeans.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(clf, f)

{'test_accuracy': 0.5053191489361702,
 'test_f1': 0.3808255659121172,
 'test_precision': 0.30620985010706636,
 'test_recall': 0.5035211267605634}
              precision    recall  f1-score   support

           0       0.70      0.51      0.59       656
           1       0.31      0.50      0.38       284

    accuracy                           0.51       940
   macro avg       0.50      0.50      0.48       940
weighted avg       0.58      0.51      0.53       940

